[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set5/set5_prob3.ipynb)


## Set 5
## 3. Word2Vec **Principles**

#### Preparation

In [43]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py

--2023-02-24 00:34:26--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0s      

2023-02-24 00:34:26 (49.6 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [44]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt

--2023-02-24 00:34:26--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2023-02-24 00:34:26 (45.0 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [45]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [46]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []

    for i in range(len(word_list)):
      word = word_list[i]
      for j in range(window_size):
        word_to_index[word_list[i]]
        right = i + j
        left = i - j
        if(left >= 0):
          trainX.append(get_word_repr(word_to_index, word))
          trainY.append(get_word_repr(word_to_index, word_list[left]))
        if(right < len(word_list)):
          trainX.append(get_word_repr(word_to_index, word))
          trainY.append(get_word_repr(word_to_index, word_list[right]))
        
          
    ##############################################################
    # TODO: Implement this function, populating trainX and trainY
    ##############################################################
    
    return np.array(trainX), np.array(trainY)

In [47]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    ##############################################################
    # TODO: 1) Create and train model in Pytorch.      
    ##############################################################

    # vocab_size = number of unique words in our text file. Will be useful 
    # when adding layers to your neural network
    vocab_size = len(word_to_index)

    model = nn.Sequential(
        nn.Linear(vocab_size, num_latent_factors).double(),
        nn.Linear(num_latent_factors, vocab_size).double()
    )
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.train()

    for epoch in range(10):
          # Erase accumulated gradients
          optimizer.zero_grad()

          # Forward pass
          output = model(torch.from_numpy(trainX))

          # Calculate loss
          loss = loss_fn(output, torch.from_numpy(trainY))

          # Backward pass
          loss.backward()
          
          # Weight update
          optimizer.step()


    ##############################################################
    # TODO: 2) Extract weights for hidden layer
    ##############################################################
    
    # set weights variable below
    weights = model[0].weight

    print("weights_hidden_shape = ", weights.shape)

    weights = (weights.detach().numpy()).T
    print("weights_output_shape = ", weights[1].shape)
    
    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [48]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
weights_hidden_shape =  torch.Size([10, 308])
weights_output_shape =  (10,)
Pair(head, must), Similarity: 0.9282510690175478
Pair(must, head), Similarity: 0.9282510690175478
Pair(something, haircut), Similarity: 0.9224151455842533
Pair(haircut, something), Similarity: 0.9224151455842533
Pair(need, found), Similarity: 0.9080686373556217
Pair(found, need), Similarity: 0.9080686373556217
Pair(tree, train), Similarity: 0.9074498512487238
Pair(train, tree), Similarity: 0.9074498512487238
Pair(pets, boat), Similarity: 0.9022582759140938
Pair(boat, pets), Similarity: 0.9022582759140938
Pair(high, low), Similarity: 0.9008839551606586
Pair(low, high), Similarity: 0.9008839551606586
Pair(socks, thank), Similarity: 0.9006436508763898
Pair(thank, socks), Similarity: 0.9006436508763898
Pair(fear, may), Similarity: 0.8974156756953375
Pair(may, fear), Similarity: 0.8974156756953375
Pair(play, tree), Similarity: 0.8960527003604988
Pair(little, cans), Similarity: 0.89